In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-22 23:35:42,432 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-22 23:35:42,433 - settings - Setting database.user to celiib
INFO - 2021-01-22 23:35:42,434 - settings - Setting database.password to newceliipass
INFO - 2021-01-22 23:35:42,439 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-22 23:35:42,440 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-22 23:35:42,457 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-22 23:35:42,734 - settings - Setting enable_python_native_blobs to True


In [3]:
#so that it will have the adapter defined
from datajoint_utils import *

In [4]:
test_mode = True

# Debugging the contains method

In [5]:
import system_utils as su

In [6]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-22 23:35:42,933 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-22 23:35:42,934 - settings - Setting database.user to celiib
INFO - 2021-01-22 23:35:42,934 - settings - Setting database.password to newceliipass
INFO - 2021-01-22 23:35:42,936 - settings - Setting enable_python_native_blobs to True


Sleeping 192 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-22 23:35:43,210 - settings - Setting enable_python_native_blobs to True


In [7]:
#(schema.jobs & "table_name='__decomposition'").delete()

# Defining the Table

In [8]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [9]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 5849


In [10]:
classified_table = minnie.BaylorManualCellType() & 'nucleus_version=3' & "(cell_type = 'excitatory') or  (cell_type = 'inhibitory')"
key_source = minnie.Decomposition & (minnie.NeuronSplitSuggestions.proj()) & (classified_table.proj()) & "n_somas<8"
key_source & "n_multi_soma_touching_limbs>0"


segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135490881732,0,0.25,30.00,0,0,1,=BLOB=,1347566,2685556,0,1,0,1,2,11,651,433,20319746.557783827,10836214.544240803,20032.079,76.397,76.397,331.736,319.774,2105,0.00010359381176403717,3.2334869431643627,7020726.017,179,0.0002998265414293264,11.76,258347147437.31827,50802762.23184045,12714.092998288601,36797.7822823104,1443280153.2811077,5975.3818
864691135474658368,0,0.25,30.00,0,0,1,=BLOB=,965673,1953444,0,2,1,1,2,17,147,25,8350798.868209888,1471003.3886958938,47491.093,318.264,271.079,531.603,502.564,4782,0.0005726398247003989,32.53061224489796,7648735.773,130,0.0006252013590115686,36.785,824134359368.1963,98927752.44496472,98689.28378882883,107747.78784716116,6339495072.063048,8255.502


In [11]:
classified_table

nucleus_id id of segmented nucleus,nucleus_version segmentation version,segment_id segment_id merged to nucleus,"cell_type morphology (valid options: 'excitatory', 'inhibitory', 'other', 'unknown')",classifier name of human classifier,timestamp
43989,3,864691135781479503,excitatory,stelios,2020-11-27 03:30:20
71096,3,864691135269778085,excitatory,stelios,2020-11-27 03:36:45
73494,3,864691135776583776,excitatory,stelios,2020-11-27 03:31:02
112738,3,864691136436394654,excitatory,stelios,2020-11-27 04:29:27
119911,3,864691136194314572,inhibitory,stelios,2020-11-27 04:15:37
120370,3,864691137196926529,excitatory,stelios,2020-11-27 03:34:29
120382,3,864691136004721098,excitatory,stelios,2020-11-27 03:08:30
122708,3,864691135523678660,inhibitory,stelios,2020-11-27 01:17:49
123799,3,864691135725629503,excitatory,stelios,2020-11-27 01:17:49
132684,3,864691135337703398,excitatory,stelios,2020-11-27 01:17:49


In [12]:
import proofreading_utils as pru
pru = reload(pru)

verbose = True

In [16]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
print(((curr_table) & "status='error'").fetch1("error_stack"))

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "<ipython-input-33-8922d3cf0196>", line 119, in make
    limb_results=split_results,
  File "/meshAfterParty/proofreading_utils.py", line 1414, in split_neuron
    verbose=verbose)
  File "/meshAfterParty/proofreading_utils.py", line 1122, in split_neuron_limbs_by_suggestions
    verbose=verbose)
  File "/meshAfterParty/proofreading_utils.py", line 950, in split_neuron_limb_by_seperated_network
    raise Exception(f"There were more not exactly one starting dictinoary: {curr_all_concept_network_data} ")
Exception: There were more not exactly one starting dictinoary: [{'starting_soma': 0, 'starting_node': 52, 'starting_endpoints': array([[ 459446., 1018820.,  744838.],
       [ 460034., 1020760.,  745755.]]), 'starting_coordinate': array([ 460034., 1020760.,  745755.]), 'touching_soma_vertices': TrackedArray([[ 459935.3, 1020870. ,

In [17]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
print(((curr_table) & "status='error'").fetch1("key"))

{'segment_id': 864691136020315128, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': Decimal('30.00'), 'process_version': 2, 'index': 0}


# Testing out the procedure

In [50]:
key = (minnie.Decomposition() & "segment_id=864691136020315128").proj().fetch1()

# 1) Get the segment id from the key
segment_id = key["segment_id"]
print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
key



# 6) For each neuron object in the list:
# - get the number of errored limbs (to indicate the success type)
# - Change the description to include the multiplicity
# - Compute the information on the largest soma faces and volume
# - Save the neuron object to the external
# - Add the new write key to a list to commit 
# 7) Write all of the keys 




---- Working on Neuron 864691136020315128 ----


{'segment_id': 864691136020315128,
 'decimation_version': 0,
 'decimation_ratio': Decimal('0.25'),
 'ver': Decimal('30.00'),
 'process_version': 2,
 'index': 0}

In [71]:
# 2) Get the decomposed neuron object from Decomposition table and the split suggestions
neuron_obj = (minnie.Decomposition & key).fetch1("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [52]:
classified_table & "segment_id=864691136020315128"

nucleus_id id of segmented nucleus,nucleus_version segmentation version,segment_id segment_id merged to nucleus,"cell_type morphology (valid options: 'excitatory', 'inhibitory', 'other', 'unknown')",classifier name of human classifier,timestamp
171640,3,864691136020315128,excitatory,stelios,2020-11-27 01:17:49


In [20]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [53]:
neuron_obj[0].all_concept_network_data

[{'starting_soma': 0,
  'starting_node': 52,
  'starting_endpoints': array([[ 459446., 1018820.,  744838.],
         [ 460034., 1020760.,  745755.]]),
  'starting_coordinate': array([ 460034., 1020760.,  745755.]),
  'touching_soma_vertices': TrackedArray([[ 459935.3, 1020870. ,  746776.8],
                [ 460893.8, 1020825. ,  746014.8],
                [ 461036.5, 1021092. ,  745491.3],
                [ 460853.2, 1020677. ,  746100.9],
                [ 460762.2, 1020734. ,  746410. ],
                [ 460792.5, 1020842. ,  746565.8],
                [ 460094.2, 1020796. ,  746745.9],
                [ 460351.5, 1021629. ,  745405.5],
                [ 460913. , 1020765. ,  745777.4],
                [ 459990.7, 1020729. ,  746590.2],
                [ 459640.8, 1020836. ,  746541. ],
                [ 460193. , 1021454. ,  745470.6],
                [ 459566.5, 1021018. ,  746453.8],
                [ 460959.1, 1020946. ,  745591.4],
                [ 460429.5, 1021590. ,  74530

In [54]:
neuron_obj[1].all_concept_network_data

[{'starting_soma': 0,
  'starting_node': 27,
  'starting_endpoints': array([[ 466830.07037037, 1032170.51851852,  755336.71851852],
         [ 469868.22660451, 1033749.13478084,  757196.29511943]]),
  'starting_coordinate': array([ 466830.07037037, 1032170.51851852,  755336.71851852]),
  'touching_soma_vertices': TrackedArray([[ 467335.4, 1031709. ,  755126.1],
                [ 466550.8, 1032402. ,  754926.5],
                [ 466894.1, 1032086. ,  755947.6],
                [ 466683. , 1032486. ,  755748. ],
                [ 467234.2, 1031866. ,  754771.5],
                [ 467211.6, 1031681. ,  755385.1],
                [ 466977. , 1032024. ,  755874. ],
                [ 466450.5, 1032508. ,  755107.1],
                [ 466689.5, 1032269. ,  754817.9],
                [ 466528.9, 1032507. ,  755224.7],
                [ 466599. , 1032444. ,  755748. ],
                [ 466546.5, 1032434. ,  755674.5],
                [ 467128.4, 1031946. ,  755738.6],
                [ 467239

In [55]:
neuron_obj[2].all_concept_network_data

[{'starting_soma': 0,
  'starting_node': 6,
  'starting_endpoints': array([[ 451977.        , 1025790.        ,  756673.        ],
         [ 452739.78275862, 1025987.4137931 ,  756100.84827586]]),
  'starting_coordinate': array([ 452739.78275862, 1025987.4137931 ,  756100.84827586]),
  'touching_soma_vertices': TrackedArray([[ 452863.1, 1026342. ,  756625.5],
                [ 452488.3, 1026026. ,  755647.2],
                [ 453061.8, 1025641. ,  756267.9],
                [ 453193.6, 1026012. ,  756544.8],
                [ 453180. , 1025787. ,  756393.8],
                [ 452518.5, 1026249. ,  756073.5],
                [ 452587.2, 1025518. ,  755591.1],
                [ 452580.6, 1025658. ,  755408.6],
                [ 452590.6, 1026128. ,  755797.9],
                [ 453274.5, 1025887. ,  756451.5],
                [ 453012. , 1025451. ,  756105. ],
                [ 452729.1, 1025391. ,  755598.1],
                [ 452886.8, 1025350. ,  756057.2],
                [ 452613.

In [56]:
neuron_obj[3].all_concept_network_data

[{'starting_soma': 0,
  'starting_node': 4,
  'starting_endpoints': array([[ 466539., 1029820.,  748191.],
         [ 487288., 1029140.,  734362.]]),
  'starting_coordinate': array([ 466539., 1029820.,  748191.]),
  'touching_soma_vertices': TrackedArray([[ 466387.1, 1029446. ,  748323.5],
                [ 465986.9, 1029331. ,  748249.4],
                [ 465995.2, 1029604. ,  747957. ],
                [ 466472.9, 1029579. ,  748665.6],
                [ 466533.4, 1029734. ,  748691.6],
                [ 466262.9, 1029446. ,  748176.2],
                [ 466096.8, 1029991. ,  748287.1],
                [ 466021.8, 1029765. ,  747919.9],
                [ 466378.6, 1030009. ,  748465.4],
                [ 466152.8, 1029944. ,  748073.2],
                [ 466421.2, 1029444. ,  748447.1],
                [ 466316. , 1029468. ,  748628. ],
                [ 466340.4, 1030034. ,  748179.8],
                [ 466207.2, 1030011. ,  748360.8],
                [ 466197. , 1029388. ,  748355

In [73]:
nru = reload(nru)
nru.clean_neuron_all_concept_network_data(neuron_obj,verbose=False)

In [74]:
neuron_obj[0].all_concept_network_data

[{'starting_soma': 0,
  'starting_node': 52,
  'starting_endpoints': array([[ 459446., 1018820.,  744838.],
         [ 460034., 1020760.,  745755.]]),
  'starting_coordinate': array([ 460034., 1020760.,  745755.]),
  'touching_soma_vertices': TrackedArray([[ 459935.3, 1020870. ,  746776.8],
                [ 460893.8, 1020825. ,  746014.8],
                [ 461036.5, 1021092. ,  745491.3],
                [ 460853.2, 1020677. ,  746100.9],
                [ 460762.2, 1020734. ,  746410. ],
                [ 460792.5, 1020842. ,  746565.8],
                [ 460094.2, 1020796. ,  746745.9],
                [ 460351.5, 1021629. ,  745405.5],
                [ 460913. , 1020765. ,  745777.4],
                [ 459990.7, 1020729. ,  746590.2],
                [ 459640.8, 1020836. ,  746541. ],
                [ 460193. , 1021454. ,  745470.6],
                [ 459566.5, 1021018. ,  746453.8],
                [ 460959.1, 1020946. ,  745591.4],
                [ 460429.5, 1021590. ,  74530

In [42]:
curr_limb.all_concept_network_data = nru.clean_all_concept_network_data(curr_limb.all_concept_network_data)
curr_limb.set_concept_network_directional?

In [30]:
neuron_obj.preprocessed_data["limb_network_stating_info"]

{0: {0: {0: {'touching_verts': TrackedArray([[ 459935.3, 1020870. ,  746776.8],
                  [ 460893.8, 1020825. ,  746014.8],
                  [ 461036.5, 1021092. ,  745491.3],
                  [ 460853.2, 1020677. ,  746100.9],
                  [ 460762.2, 1020734. ,  746410. ],
                  [ 460792.5, 1020842. ,  746565.8],
                  [ 460094.2, 1020796. ,  746745.9],
                  [ 460351.5, 1021629. ,  745405.5],
                  [ 460913. , 1020765. ,  745777.4],
                  [ 459990.7, 1020729. ,  746590.2],
                  [ 459640.8, 1020836. ,  746541. ],
                  [ 460193. , 1021454. ,  745470.6],
                  [ 459566.5, 1021018. ,  746453.8],
                  [ 460959.1, 1020946. ,  745591.4],
                  [ 460429.5, 1021590. ,  745306.5],
                  [ 459824.4, 1020808. ,  746601.2],
                  [ 459756.2, 1020884. ,  746678.9],
                  [ 460926.8, 1021006. ,  745405. ],
                  [

In [28]:
neuron_obj.preprocessed_data["limb_concept_networks"]

{0: {0: [<networkx_utils.GraphOrderedEdges at 0x7fcb44641940>,
   <networkx_utils.GraphOrderedEdges at 0x7fcb598c5550>]},
 1: {0: [<networkx_utils.GraphOrderedEdges at 0x7fcb598c5438>]},
 2: {0: [<networkx_utils.GraphOrderedEdges at 0x7fcb598c55f8>]},
 3: {0: [<networkx_utils.GraphOrderedEdges at 0x7fcb598c5630>]},
 4: {0: [<networkx_utils.GraphOrderedEdges at 0x7fcb598c55c0>]}}

In [32]:
curr_limb.all_concept_network_data[0].keys()

dict_keys(['starting_soma', 'starting_node', 'starting_endpoints', 'starting_coordinate', 'touching_soma_vertices', 'soma_group_idx', 'concept_network'])

In [43]:
curr_limb.all_concept_network_data

[{'starting_soma': 0,
  'starting_node': 52,
  'starting_endpoints': array([[ 459446., 1018820.,  744838.],
         [ 460034., 1020760.,  745755.]]),
  'starting_coordinate': array([ 460034., 1020760.,  745755.]),
  'touching_soma_vertices': TrackedArray([[ 459935.3, 1020870. ,  746776.8],
                [ 460893.8, 1020825. ,  746014.8],
                [ 461036.5, 1021092. ,  745491.3],
                [ 460853.2, 1020677. ,  746100.9],
                [ 460762.2, 1020734. ,  746410. ],
                [ 460792.5, 1020842. ,  746565.8],
                [ 460094.2, 1020796. ,  746745.9],
                [ 460351.5, 1021629. ,  745405.5],
                [ 460913. , 1020765. ,  745777.4],
                [ 459990.7, 1020729. ,  746590.2],
                [ 459640.8, 1020836. ,  746541. ],
                [ 460193. , 1021454. ,  745470.6],
                [ 459566.5, 1021018. ,  746453.8],
                [ 460959.1, 1020946. ,  745591.4],
                [ 460429.5, 1021590. ,  74530

In [46]:
st_soma = curr_limb.all_concept_network_data[0]["starting_soma"]
st_node = curr_limb.all_concept_network_data[0]["starting_node"]
curr_limb.set_concept_network_directional(starting_soma=st_soma,
                                         starting_node=st_node)


In [48]:
nru.clean_neuron_all_concept_network_data(neuron_obj)

In [49]:
neuron_obj

In [ ]:
def clean_neuron_all_concept_network_data(neuron_obj):
    
    for curr_limb in neuron_obj:
        curr_limb.all_concept_network = nru.clean_all_concept_network_data(curr_limb.all_concept_network_data)
        
        #setting the concept network
        st_soma = curr_limb.all_concept_network_data[0]["starting_soma"]
        st_node = curr_limb.all_concept_network_data[0]["starting_node"]
        curr_limb.set_concept_network_directional(starting_soma=st_soma,
                                                 starting_node=st_node)
        

In [39]:
nru = reload(nru)
nru.clean_all_concept_network_data(curr_limb.all_concept_network_data)

[{'starting_soma': 0,
  'starting_node': 52,
  'starting_endpoints': array([[ 459446., 1018820.,  744838.],
         [ 460034., 1020760.,  745755.]]),
  'starting_coordinate': array([ 460034., 1020760.,  745755.]),
  'touching_soma_vertices': TrackedArray([[ 459935.3, 1020870. ,  746776.8],
                [ 460893.8, 1020825. ,  746014.8],
                [ 461036.5, 1021092. ,  745491.3],
                [ 460853.2, 1020677. ,  746100.9],
                [ 460762.2, 1020734. ,  746410. ],
                [ 460792.5, 1020842. ,  746565.8],
                [ 460094.2, 1020796. ,  746745.9],
                [ 460351.5, 1021629. ,  745405.5],
                [ 460913. , 1020765. ,  745777.4],
                [ 459990.7, 1020729. ,  746590.2],
                [ 459640.8, 1020836. ,  746541. ],
                [ 460193. , 1021454. ,  745470.6],
                [ 459566.5, 1021018. ,  746453.8],
                [ 460959.1, 1020946. ,  745591.4],
                [ 460429.5, 1021590. ,  74530

In [25]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict=dict(L0="all"),
                     soma_border_vertices=True,
                     soma_border_vertices_size=0.05)


 Working on visualization type: mesh

 Working on visualization type: skeleton
working on soma border vertices


In [22]:
pru.multi_soma_split_suggestions(neuron_obj,
                                 plot_intermediates=True,
                                 plot_suggestions=True,
                                verbose=True,
                                )

multi_soma_limbs = []
multi_touch_limbs = [0]


 -------- Working on limb 0------------
Starting combinations to process = []
Total number of graphs at the end of the split = 1: [array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56])]


-------- Suggestions for Limb 0------


 No suggested cuts for this limb!!

 Working on visualization type: mesh

 Working on visualization type: skeleton


{0: []}

In [ ]:
# 3) Retrieve the multi soma suggestions
split_results = (minnie.NeuronSplitSuggestions & key).fetch1("split_results")

In [ ]:
# 4) Get the number of splits required for this neuron
n_paths_cut = pru.get_n_paths_cut(split_results)

if verbose:
    print(f"n_paths_cut = {n_paths_cut}")

In [ ]:
neuron = reload(neuron)
neuron_obj = neuron.Neuron(neuron_obj)

In [ ]:
# 5) Split the neuron into a list of neuron objects
neuron_list = pru.split_neuron(neuron_obj,
                limb_results=split_results,
                                    )

In [ ]:
"""
# 6) For each neuron object in the list:
# - get the number of errored limbs (to indicate the success type)
# - Compute the information on the largest soma faces and volume
# - Save the neuron object to the external
# - Add the new write key to a list to commit 
"""
whole_pass_time = time.time()

neuron_idx = 0
n = neuron_list[neuron_idx]
#for n in neuron_list:
#     nviz.visualize_neuron(n,
#                          limb_branch_dict="all")

# - get the number of errored limbs (to indicate the success type)
if n.n_error_limbs == 0:
    split_success = 0
elif n.multi_soma_touching_limbs == 0:
    split_successs = 1
elif n.same_soma_multi_touching_limbs == 0:
    split_success = 2
else:
    split_success = 3
    
if verbose:
    print(f"split_success = {split_success}")

# - Compute the information on the largest soma faces and volume
soma_volumes = [n[k].volume for k in n.get_soma_node_names()] 
soma_n_faces = [len(n[k].mesh.faces) for k in n.get_soma_node_names()] 

largest_n_faces = np.max(soma_n_faces)
largest_volume = np.max(soma_volumes)

if verbose:
    print(f"largest_n_faces = {largest_n_faces}")
    print(f"largest_volume = {largest_volume}")
    
    
#6) Save the file in a certain location
if False:
    save_time = time.time()
    ret_file_path = n.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                      return_file_path=True,
                                     export_mesh=False,
                                     suppress_output=True)

    ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
    print(f"Save time = {time.time() - save_time}")
else:
    ret_file_path_str = "dummy_holder"


    
#7) Pass stats and file location to insert
new_key = dict(key,
               split_index = neuron_idx,
               multiplicity=len(neuron_list),
               
               n_splits = n_paths_cut,
               split_success = split_success,
               
               max_soma_n_faces = largest_n_faces,
               max_soma_volume = largest_volume,
               
               
               decomposition=ret_file_path_str,
               n_vertices=len(n.mesh.vertices),
               n_faces=len(n.mesh.faces),
               run_time=np.round(time.time() - whole_pass_time,4)
              )

stats_dict = n.neuron_stats()
new_key.update(stats_dict)

new_
    


In [ ]:
neuron = reload(neuron)
n = neuron.Neuron(n)

In [ ]:
n["S0"].volume

In [ ]:
n["S0"].volume

In [ ]:
# 6) For each neuron object in the list:
# - get the number of errored limbs (to indicate the success type)
# - Change the description to include the multiplicity
# - Compute the information on the largest soma faces and volume
# - Save the neuron object to the external
# - Add the new write key to a list to commit 

In [ ]:
import numpy as np
import time

max_n_somas=8



@schema
class DecompositionSplit(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    process_version : int unsigned #the version of the preprocessing pipeline run
    index : tinyint unsigned  #the index of the neuron object that resulted from that mesh (indexed starting at 0)
    split_index: : tinyint unsigned  #the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    n_splits             : int unsigned                 # the number of cuts required to help split the neuron
    split_success        : tinyint unsigned             # the successfulness of the splitting
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    max_soma_n_faces:  int unsigned                 # The largest number of faces of the somas
    max_soma_volume:  int unsigned                 # The largest volume of the somas
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """
                             
    
    classified_table = minnie.BaylorManualCellType() & 'nucleus_version=3'
    key_source = minnie.Decomposition & (minnie.NeuronSplitSuggestions.proj()) & (classified_table.proj()) & f"n_somas<{max_n_somas}"
    key_source
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decomposed neurong object from Decomposition table
        3) Run the multi_soma split suggestions algorithm
        4) Get the number of splits required for this neuron
        5) Split the neuron into a list of neuron objects
        6) For each neuron object in the list:
        - get the number of errored limbs (to indicate the success type)
        - Change the description to include the multiplicity
        - Compute the information on the largest soma faces and volume
        - Save the neuron object to the external
        - Add the new write key to a list to commit 
        7) Write all of the keys 
        """
        
        whole_pass_time = time.time()
        
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        global_start = time.time()
        
        #2) Get the decomposed neurong object from Decomposition table
        neuron_obj = minnie.
        

        #3) Get the somas info 
        somas = du.get_soma_mesh_list(segment_id) 
        soma_ver = du.get_soma_mesh_list_ver(segment_id)
                  
        
        print(f"somas = {somas}")
                  
        #3b) Get the glia and nuclei information 
        glia_faces,nuclei_faces = du.get_segment_glia_nuclei_faces(segment_id,return_empty_list=True)
        
                  
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"],
        glia_faces=glia_faces,
        nuclei_faces = nuclei_faces,

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       ver = soma_ver,
                       process_version = process_version,
                       index = 0,
                       multiplicity=1,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table).delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")